# 11. 워드 임베딩의 평균

특정 문장 내의 단어들의 임베딩 벡터들의 평균이 그 문장의 벡터가 될 수 있다. 이번 챕터에서는 임베딩이 잘 된 상황에서는 단어 벡터들의 평균만으로도 텍스트 분류를 할 수 있음을 보이고, 워드 임베딩의 중요성을 이해하도록 한다.

영화 사이트 IMDB 영화 리뷰 데이터는 리뷰 텍스트에 리뷰가 긍정인 경우 1을, 부정인 경우 0으로 레이블링 한 데이터로 25,000개의 훈련 데이터와 테스트 데이터 25,000개로 구성된 데이터이다. 케라스를 통해서 이 데이터셋을 바로 다운로드 할 수 있다.

## 1) 데이터 로드와 전처리

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import imdb

케라스에서 imdb.data_load()를 통해서 IMDB 리뷰 데이터를 다운로드할 수 있다.

데이터를 로드할 때 파라메터로 NUM_WORDS를 사용하면 이 데이터에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지 사용할 것인지를 의미한다.

만약 vocab_size를 20,000으로 설정하면 등장 빈도 순위가 20,000등이 넘는 단어들은 전체 데이터에서 전부 제거 후 로드한다.

In [2]:
vocab_size = 20000

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 :',len(x_train))
print('테스트용 리뷰 개수 :',len(x_test))

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/dlpc/python/lib/python3.6/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


/usr/local/dlpc/python/lib/python3.6/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


이 데이터는 이미 정수 인코딩까지의 전처리가 완료된 상태이다. 그래서 각 단어를 정수로 인코딩하는 과정을 할 필요가 없다.

In [3]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


위에 보이는 것과 같이 정수 시퀀스가 출력된다.

In [5]:
print(y_train[0]) # 첫번째 리뷰의 레이블. 1은 긍정 리뷰를 의미한다.

1


In [7]:
# 각 리뷰의 평균 길이 계산
print('훈련용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('테스트용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

훈련용 리뷰의 평균 길이: 238
테스트용 리뷰의 평균 길이: 230


In [8]:
# 평균보다 큰 수치인 400으로 두 리뷰 데이터를 패딩
max_len = 400

x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)
print('x_train의 크기(shape) :', x_train.shape)
print('x_test의 크기(shape) :', x_test.shape)

x_train의 크기(shape) : (25000, 400)
x_test의 크기(shape) : (25000, 400)


## 2) 모델 설계

모델의 입력으로 사용하기 위한 전처리를 다 마치면, Embedding() 다음에 GlobalAveragePooling1D()을 추가하면 해당 문장의 모든 단어 벡터들의 평균 벡터를 구하게 된다.

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_len))
model.add(GlobalAveragePooling1D()) # 모든 단어 벡터의 평균을 구한다.
model.add(Dense(1, activation='sigmoid')) # 이진 분류를 수행하므로 활성화 함수를 시그모이드 함수로 사용하는 뉴런 1개를 배치한다.

In [11]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('embedding_average_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [12]:
# 훈련 데이터의 20%를 검증 데이터로 사용하고 총 10회 학습한다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 11s 17ms/step - loss: 0.6728 - acc: 0.6280 - val_loss: 0.5393 - val_acc: 0.8018

Epoch 00001: val_acc improved from -inf to 0.80180, saving model to embedding_average_model.h5
Epoch 2/10
625/625 [==============================] - 10s 17ms/step - loss: 0.4864 - acc: 0.8405 - val_loss: 0.3962 - val_acc: 0.8656

Epoch 00002: val_acc improved from 0.80180 to 0.86560, saving model to embedding_average_model.h5
Epoch 3/10
625/625 [==============================] - 10s 16ms/step - loss: 0.3495 - acc: 0.8886 - val_loss: 0.3364 - val_acc: 0.8780

Epoch 00003: val_acc improved from 0.86560 to 0.87800, saving model to embedding_average_model.h5
Epoch 4/10
625/625 [==============================] - 9s 15ms/step - loss: 0.2867 - acc: 0.9086 - val_loss: 0.3078 - val_acc: 0.8806

Epoch 00004: val_acc improved from 0.87800 to 0.88060, saving model to embedding_average_model.h5
Epoch 5/10
625/625 [==============================] - 10s 16ms/step - lo

In [13]:
# 학습 후 테스트 데이터에 대한 평가 시행
loaded_model = load_model('embedding_average_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 1s 2ms/step - loss: 0.2819 - acc: 0.8870

 테스트 정확도: 0.8870
